In [15]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime

# 크롤링 함수를 정의
def crawl_kbo_schedule(year, month):
    # 웹 드라이버 실행 및 페이지 열기
    url = "https://www.koreabaseball.com/Schedule/Schedule.aspx?seriesId=0,9,6"  # 정규 시즌
    driver = webdriver.Chrome()  # 경로 수정
    driver.get(url)
    
    try:
        # 년도와 월 선택 요소가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlYear")))

        # 년도와 월 선택
        year_dropdown = Select(driver.find_element(By.ID, "ddlYear"))
        year_dropdown.select_by_value(year)

        month_dropdown = Select(driver.find_element(By.ID, "ddlMonth"))
        month_dropdown.select_by_value(month)

        # 경기 일정 테이블이 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tbl-type06")))
        table = driver.find_element(By.CLASS_NAME, "tbl-type06")
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # 데이터를 저장할 리스트
        data_lst = []
        game_data_by_date = {}  # 날짜별 경기를 저장할 딕셔너리
        
        # 경기 일정 정보 처리
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            data = [column.text for column in columns]

            if len(data) == 0:
                continue  # 빈 행 건너뛰기
            elif len(data) == 9:
                previous_date = data[0]  # 날짜 저장
            elif len(data) == 8:
                data.insert(0, previous_date)  # 날짜가 생략된 행에 이전 날짜 삽입
            
            # 데이터 길이를 체크하여 처리
            if len(data) < 9:
                print("예상치 못한 데이터 형식:", data)
                continue  # 잘못된 데이터 형식은 건너뛰기

            # 날짜 및 경기 정보 가져오기
            raw_date = data[0]  # 날짜 형식: "08.02(금)" 
            game_time = data[1]
            game_info = data[2]
            tv = data[5]
            stadium = data[7]
            note = data[8]

            # 날짜 형식을 "yyyy-mm-dd"로 변환
            try:
                # 날짜에서 "(요일)"을 제거하고, "mm.dd" 형식을 사용해 datetime 객체로 변환
                date_str = raw_date.split('(')[0].strip()  # "08.02"로 변환
                date = datetime.strptime(date_str, "%m.%d").replace(year=int(year)).strftime("%Y-%m-%d")
            except ValueError:
                continue  # 날짜 형식이 맞지 않으면 해당 데이터는 건너뛰기

            # 경기 정보 분리 (team1 vs team2)
            if 'vs' in game_info:
                teams_score = game_info.split('vs')
                team1_info = teams_score[0]
                team2_info = teams_score[1]

                # 팀1과 점수 분리
                team1 = ''.join([i for i in team1_info if not i.isdigit()]).strip()
                team1_score = ''.join([i for i in team1_info if i.isdigit()])

                # 팀2와 점수 분리
                team2 = ''.join([i for i in team2_info if not i.isdigit()]).strip()
                team2_score = ''.join([i for i in team2_info if i.isdigit()])
            else:
                continue  # 경기 정보가 제대로 분리되지 않은 경우 건너뛰기

            # 취소된 경기 처리
            is_cancelled = False
            if '취소' in note:
                is_cancelled = True
                cancel_reason = note
                # 팀 이름은 그대로 유지하고 점수만 "-"로 변경
                team1_score = "-"
                team2_score = "-"
            else:
                cancel_reason = "-"

                # 점수 정보가 없는 경우 "-"로 설정
                if not team1_score:
                    team1_score = "-"
                if not team2_score:
                    team2_score = "-"

            # 경기 결과 처리 (0: 승리, 1: 패배, 2: 무승부, -: 취소)
            if team1_score != "-" and team2_score != "-":
                if int(team1_score) > int(team2_score):
                    result = "0"  # team1 승리
                elif int(team1_score) < int(team2_score):
                    result = "1"  # team1 패배
                else:
                    result = "2"  # 무승부
            else:
                result = "-"

            # 날짜별 경기 저장
            if date not in game_data_by_date:
                game_data_by_date[date] = []
            game_data_by_date[date].append({
                "date": date,
                "team1": team1,
                "team1_score": team1_score,
                "team2": team2,
                "team2_score": team2_score,
                "result": result,
                "stadium": stadium,
                "cancel": is_cancelled,
                "cancelReason": cancel_reason,
                "game_time": game_time  # 시간을 저장해서 더블헤더 여부 판단
            })

        # 더블헤더 여부 처리
        for date, games in game_data_by_date.items():
            # 같은 날짜에 같은 팀이 두 번 경기를 하는 경우 더블헤더 처리
            team_count = {}
            for game in games:
                team1 = game['team1']
                if team1 in team_count:
                    team_count[team1] += 1
                else:
                    team_count[team1] = 1

            for game in games:
                team1 = game['team1']
                if team_count[team1] > 1:
                    # 같은 날짜에 같은 팀이 두 번 출전하는 경우 (더블헤더)
                    if game['game_time'] == min([g['game_time'] for g in games if g['team1'] == team1]):
                        double_header_order = 0  # 앞 경기
                    else:
                        double_header_order = 1  # 뒷 경기
                else:
                    double_header_order = -1  # 더블헤더 아님
                
                # 더블헤더 여부를 추가하여 최종 데이터 저장
                game_data_by_date[date][games.index(game)]["doubleHeaderGameOrder"] = double_header_order
                data_lst.append(game_data_by_date[date][games.index(game)])

        return data_lst

    except TimeoutException:
        # 경기 일정 테이블 자체가 없는 경우 (경기가 없는 달)
        print(f"{year}년 {month}월에 경기가 없습니다.")
        return []

    finally:
        # 크롤링 완료 후 브라우저 종료
        driver.quit()

# 크롤링 함수 실행 예시 (2024년 8월 데이터)
schedule_data = crawl_kbo_schedule("2024", "09")

# 결과를 데이터프레임으로 변환
if schedule_data:
    df = pd.DataFrame(schedule_data)
    # CSV 파일로 저장
    df.to_csv('kbo_schedule.csv', index=False)
    print("CSV 파일이 생성되었습니다.")
else:
    print("해당 기간에 데이터를 찾을 수 없습니다.")


CSV 파일이 생성되었습니다.


In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

# 크롤링 함수를 정의
def crawl_kbo_schedule(year, month):
    # 웹 드라이버 실행 및 페이지 열기
    url = "https://www.koreabaseball.com/Schedule/Schedule.aspx"
    driver = webdriver.Chrome()  # 경로 수정
    driver.get(url)
    time.sleep(3)

    # 년도와 월 선택
    year_dropdown = Select(driver.find_element(By.ID, "ddlYear"))
    year_dropdown.select_by_value(year)
    month_dropdown = Select(driver.find_element(By.ID, "ddlMonth"))
    month_dropdown.select_by_value(month)
    time.sleep(5)

    try:
        # 경기 일정 테이블 찾기
        table = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "tbl-type06"))
        )
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # 데이터를 저장할 리스트
        data_lst = []
        previous_date = None
        
        # 경기 일정 정보 처리
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            data = [column.text for column in columns]
            print(data)  # 디버깅을 위한 출력

            if len(data) == 0:
                continue  # 빈 행 건너뛰기
            elif len(data) == 9:
                previous_date = data[0]  # 날짜 저장  
            elif len(data) == 8:
                data.insert(0, previous_date)
                
            date = data[0]
            game_time = data[1]
            game_info = data[2] 
            tv = data[5] 
            stadium = data[7] 
            note = data[8] 

            # 경기 정보 분리 (팀1 팀1점수 팀2점수 팀2)
            if 'vs' in game_info:
                teams_score = game_info.split('vs')
                team1_info = teams_score[0]
                team2_info = teams_score[1]

                # 팀1과 점수 분리
                team1 = ''.join([i for i in team1_info if not i.isdigit()])
                team1_score = ''.join([i for i in team1_info if i.isdigit()])

                # 팀2와 점수 분리
                team2 = ''.join([i for i in team2_info if not i.isdigit()])
                team2_score = ''.join([i for i in team2_info if i.isdigit()])
            else:
                team1 = team1_score = team2 = team2_score = None

            # 취소된 경기는 점수를 100으로 설정
            if not team1_score or not team2_score:
                team1_score = 100
                team2_score = 100

            # 리스트에 저장
            data_lst.append([date, game_time, team1, team1_score, team2, team2_score, tv, stadium, note])

        return data_lst

    except TimeoutException:
        # 경기 일정 테이블 자체가 없는 경우 (경기가 없는 달)
        print(f"{year}년 {month}월에 경기가 없습니다.")
        return [[pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA, pd.NA]]

    finally:
        # 크롤링 완료 후 브라우저 종료
        driver.quit()

# 크롤링 함수 실행 예시 (2024년 8월 데이터)
schedule_data = crawl_kbo_schedule("2024", "08")

# 결과를 데이터프레임으로 변환
columns = ['날짜', '시간', '팀1', '팀1점수', '팀2', '팀2점수', 'TV', '구장', '비고']
df = pd.DataFrame(schedule_data, columns=columns)

# CSV 파일로 저장
df.to_csv('kbo_schedule.csv', index=False)

print("CSV 파일이 생성되었습니다.")

해당 기간에 데이터를 찾을 수 없습니다.


In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from datetime import datetime

# 크롤링 함수를 정의
def crawl_kbo_postseason(year, month):
    # 웹 드라이버 실행 및 페이지 열기
    url = "https://www.koreabaseball.com/Schedule/Schedule.aspx?seriesId=3,4,5,7"  # 포스트 시즌
    driver = webdriver.Chrome()  # 경로 수정
    driver.get(url)
    
    try:
        # 년도와 월 선택 요소가 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "ddlYear")))

        # 년도와 월 선택
        year_dropdown = Select(driver.find_element(By.ID, "ddlYear"))
        year_dropdown.select_by_value(year)

        month_dropdown = Select(driver.find_element(By.ID, "ddlMonth"))
        month_dropdown.select_by_value(month)

        # 경기 일정 테이블이 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tbl-type06")))
        table = driver.find_element(By.CLASS_NAME, "tbl-type06")
        rows = table.find_elements(By.TAG_NAME, "tr")
        
        # 데이터를 저장할 리스트
        data_lst = []
        game_data_by_date = {}  # 날짜별 경기를 저장할 딕셔너리
        
        # 경기 일정 정보 처리
        for row in rows:
            columns = row.find_elements(By.TAG_NAME, "td")
            data = [column.text for column in columns]

            if len(data) == 0:
                continue  # 빈 행 건너뛰기
            elif len(data) == 9:
                previous_date = data[0]  # 날짜 저장
            elif len(data) == 8:
                data.insert(0, previous_date)  # 날짜가 생략된 행에 이전 날짜 삽입
            
            # 데이터 길이를 체크하여 처리
            if len(data) < 9:
                print("예상치 못한 데이터 형식:", data)
                continue  # 잘못된 데이터 형식은 건너뛰기

            # 날짜 및 경기 정보 가져오기
            raw_date = data[0]  # 날짜 형식: "08.02(금)" 
            game_time = data[1]
            game_info = data[2]
            tv = data[5]
            stadium = data[7]
            note = data[8]

            # 날짜 형식을 "yyyy-mm-dd"로 변환
            try:
                # 날짜에서 "(요일)"을 제거하고, "mm.dd" 형식을 사용해 datetime 객체로 변환
                date_str = raw_date.split('(')[0].strip()  # "08.02"로 변환
                date = datetime.strptime(date_str, "%m.%d").replace(year=int(year)).strftime("%Y-%m-%d")
            except ValueError:
                continue  # 날짜 형식이 맞지 않으면 해당 데이터는 건너뛰기

            # 경기 정보 분리 (team1 vs team2)
            if 'vs' in game_info:
                teams_score = game_info.split('vs')
                team1_info = teams_score[0]
                team2_info = teams_score[1]

                # 팀1과 점수 분리
                team1 = ''.join([i for i in team1_info if not i.isdigit()]).strip()
                team1_score = ''.join([i for i in team1_info if i.isdigit()])

                # 팀2와 점수 분리
                team2 = ''.join([i for i in team2_info if not i.isdigit()]).strip()
                team2_score = ''.join([i for i in team2_info if i.isdigit()])
            else:
                continue  # 경기 정보가 제대로 분리되지 않은 경우 건너뛰기

            # 취소된 경기 처리
            is_cancelled = False
            if '취소' in note:
                is_cancelled = True
                cancel_reason = note
                # 팀 이름은 그대로 유지하고 점수만 "-"로 변경
                team1_score = "-"
                team2_score = "-"
            else:
                cancel_reason = "-"

                # 점수 정보가 없는 경우 "-"로 설정
                if not team1_score:
                    team1_score = "-"
                if not team2_score:
                    team2_score = "-"

            # 경기 결과 처리 (0: 승리, 1: 패배, 2: 무승부, -: 취소)
            if team1_score != "-" and team2_score != "-":
                if int(team1_score) > int(team2_score):
                    result = "0"  # team1 승리
                elif int(team1_score) < int(team2_score):
                    result = "1"  # team1 패배
                else:
                    result = "2"  # 무승부
            else:
                result = "-"

            # 날짜별 경기 저장
            if date not in game_data_by_date:
                game_data_by_date[date] = []
            game_data_by_date[date].append({
                "date": date,
                "team1": team1,
                "team1_score": team1_score,
                "team2": team2,
                "team2_score": team2_score,
                "result": result,
                "stadium": stadium,
                "cancel": is_cancelled,
                "cancelReason": cancel_reason,
                "game_time": game_time  # 시간을 저장해서 더블헤더 여부 판단
            })

        # 더블헤더 여부 처리
        for date, games in game_data_by_date.items():
            # 같은 날짜에 같은 팀이 두 번 경기를 하는 경우 더블헤더 처리
            team_count = {}
            for game in games:
                team1 = game['team1']
                if team1 in team_count:
                    team_count[team1] += 1
                else:
                    team_count[team1] = 1

            for game in games:
                team1 = game['team1']
                if team_count[team1] > 1:
                    # 같은 날짜에 같은 팀이 두 번 출전하는 경우 (더블헤더)
                    if game['game_time'] == min([g['game_time'] for g in games if g['team1'] == team1]):
                        double_header_order = 0  # 앞 경기
                    else:
                        double_header_order = 1  # 뒷 경기
                else:
                    double_header_order = -1  # 더블헤더 아님
                
                # 더블헤더 여부를 추가하여 최종 데이터 저장
                game_data_by_date[date][games.index(game)]["doubleHeaderGameOrder"] = double_header_order
                data_lst.append(game_data_by_date[date][games.index(game)])

        return data_lst

    except TimeoutException:
        # 경기 일정 테이블 자체가 없는 경우 (경기가 없는 달)
        print(f"{year}년 {month}월에 경기가 없습니다.")
        return []

    finally:
        # 크롤링 완료 후 브라우저 종료
        driver.quit()

# 데이터를 CSV로 저장하는 함수
def save_to_csv(data, year):
    if data:
        df = pd.DataFrame(data)
        file_name = f"kbo_postseason_{year}.csv"
        df.to_csv(file_name, index=False)
        print(f"{year}년 포스트 시즌 데이터가 {file_name}로 저장되었습니다.")
    else:
        print(f"{year}년에 저장할 데이터가 없습니다.")

# 2010년부터 2024년까지 8월부터 11월까지 포스트 시즌 데이터를 수집하여 CSV로 저장
for year in range(2010, 2025):
    all_data = []
    for month in range(8, 12):  # 8월부터 11월까지
        print(f"{year}년 {month}월 포스트 시즌 데이터를 수집합니다.")
        schedule_data = crawl_kbo_postseason(str(year), f"{month:02d}")
        all_data.extend(schedule_data)
    
    # 수집된 데이터를 해당 연도에 맞는 CSV 파일로 저장
    save_to_csv(all_data, year)


2010년 8월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2010년 9월 포스트 시즌 데이터를 수집합니다.
2010년 10월 포스트 시즌 데이터를 수집합니다.
2010년 11월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2010년 포스트 시즌 데이터가 kbo_postseason_2010.csv로 저장되었습니다.
2011년 8월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2011년 9월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2011년 10월 포스트 시즌 데이터를 수집합니다.
2011년 11월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2011년 포스트 시즌 데이터가 kbo_postseason_2011.csv로 저장되었습니다.
2012년 8월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2012년 9월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2012년 10월 포스트 시즌 데이터를 수집합니다.
2012년 11월 포스트 시즌 데이터를 수집합니다.
2012년 포스트 시즌 데이터가 kbo_postseason_2012.csv로 저장되었습니다.
2013년 8월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2013년 9월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2013년 10월 포스트 시즌 데이터를 수집합니다.
2013년 11월 포스트 시즌 데이터를 수집합니다.
2013년 포스트 시즌 데이터가 kbo_postseason_2013.csv로 저장되었습니다.
2014년 8월 포스트 시즌 데이터를 수집합니다.
예상치 못한 데이터 형식: ['데이터가 없습니다.']
2014년 9월

In [6]:
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import schedule

# 크롤링 함수
def crawl_kbo_team_winrate():
    """
    KBO 정규 시즌 팀별 승률 데이터를 크롤링하여 콘솔에 출력하는 함수
    """
    url = "https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx"  # 팀 순위 페이지

    # 웹 드라이버 실행 및 페이지 열기
    driver = webdriver.Chrome()  # 경로 수정 필요a
    driver.get(url)

    try:
        # 순위 테이블이 로드될 때까지 대기
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tData")))

        # 순위 테이블 추출
        table = driver.find_element(By.CLASS_NAME, "tData")
        rows = table.find_elements(By.TAG_NAME, "tr")

        # 데이터를 저장할 리스트
        data_lst = []

        # 팀별 데이터 추출
        for row in rows[1:]:  # 첫 번째 행은 헤더이므로 건너뛰기
            columns = row.find_elements(By.TAG_NAME, "td")

            if len(columns) < 7:
                continue  # 팀명이나 승률 데이터가 없으면 건너뛰기

            team = columns[1].text.strip()  # 팀명
            win_rate = columns[6].text.strip()  # 승률

            data_lst.append({
                "team": team,
                "win_rate": win_rate,
                "date": datetime.now().strftime("%Y-%m-%d %H:%M")  # 데이터 수집 날짜 및 시간 추가
            })

        # 크롤링한 데이터를 출력
        if data_lst:
            print("크롤링한 팀별 승률 데이터:")
            for data in data_lst:
                print(f"팀: {data['team']}, 승률: {data['win_rate']}, 수집 시간: {data['date']}")
        else:
            print("저장할 데이터가 없습니다.")

    except Exception as e:
        print(f"크롤링 중 오류 발생: {e}")

    finally:
        # 크롤링 완료 후 브라우저 종료
        driver.quit()

# 1분마다 실행되도록 스케줄링 (실험용)
schedule.every(1).minutes.do(crawl_kbo_team_winrate)

# 스케줄러가 실행되도록 루프
while True:
    schedule.run_pending()
    time.sleep(1)


크롤링 중 오류 발생: Message: 
Stacktrace:
0   chromedriver                        0x0000000100910248 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x0000000100908730 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x000000010051c260 cxxbridge1$string$len + 89488
3   chromedriver                        0x000000010056050c cxxbridge1$string$len + 368700
4   chromedriver                        0x000000010059a7d0 cxxbridge1$string$len + 606976
5   chromedriver                        0x000000010055512c cxxbridge1$string$len + 322652
6   chromedriver                        0x0000000100555d7c cxxbridge1$string$len + 325804
7   chromedriver                        0x00000001008d84d8 cxxbridge1$str$ptr + 1678560
8   chromedriver                        0x00000001008dce40 cxxbridge1$str$ptr + 1697352
9   chromedriver                        0x00000001008bd5ec cxxbridge1$str$ptr + 1568244
10  chromedriver                        0x00000001008dd710 cxxbridge1$str$pt

KeyboardInterrupt: 